### モデルの作成
---
[参考](http://ni4muraano.hatenablog.com/entry/2017/08/10/101053)

In [ ]:
from keras.models import Model
from keras.layers import Input
from keras.layers.convolutional import Conv2D, ZeroPadding2D, Conv2DTranspose
from keras.layers.merge import concatenate
from keras.layers import LeakyReLU, BatchNormalization, Activation, Dropout

class UNet(object):
    def __init__(self, input_channel_count, output_channel_count, first_layer_filter_count):
        self.INPUT_IMAGE_SIZE = 512
        self.CONCATENATE_AXIS = -1
        self.CONV_FILTER_SIZE = 4
        self.CONV_STRIDE = 2
        self.CONV_PADDING = (1, 1)
        self.DECONV_FILTER_SIZE = 2
        self.DECONV_STRIDE = 2

        # (512 x 512 x input_channel_count)
        inputs = Input((self.INPUT_IMAGE_SIZE, self.INPUT_IMAGE_SIZE, input_channel_count))

        # エンコーダーの作成
        # (256 x 256 x N)
        enc1 = ZeroPadding2D(self.CONV_PADDING)(inputs)
        enc1 = Conv2D(first_layer_filter_count, self.CONV_FILTER_SIZE, strides=self.CONV_STRIDE)(enc1)

        # (128 x 128 x 2N)
        filter_count = first_layer_filter_count*2
        enc2 = self._add_encoding_layer(filter_count, enc1)

        # (64 x 64 x 4N)
        filter_count = first_layer_filter_count*4
        enc3 = self._add_encoding_layer(filter_count, enc2)

        # (32 x 32 x 8N)
        filter_count = first_layer_filter_count*8
        enc4 = self._add_encoding_layer(filter_count, enc3)

        # (16 x 16 x 8N)
        enc5 = self._add_encoding_layer(filter_count, enc4)

        # (8 x 8 x 8N)
        enc6 = self._add_encoding_layer(filter_count, enc5)

        # (4 x 4 x 8N)
        enc7 = self._add_encoding_layer(filter_count, enc6)

        # (2 x 2 x 8N)
        enc8 = self._add_encoding_layer(filter_count, enc7)

        # (1 x 1 x 8N)
        enc9 = self._add_encoding_layer(filter_count, enc8)
        
        # デコーダーの作成
        # (2 x 2 x 8N)
        dec1 = self._add_decoding_layer(filter_count, True, enc9)
        dec1 = concatenate([dec1, enc8], axis=self.CONCATENATE_AXIS)

        # (4 x 4 x 8N)
        dec2 = self._add_decoding_layer(filter_count, True, dec1)
        dec2 = concatenate([dec2, enc7], axis=self.CONCATENATE_AXIS)

        # (8 x 8 x 8N)
        dec3 = self._add_decoding_layer(filter_count, True, dec2)
        dec3 = concatenate([dec3, enc6], axis=self.CONCATENATE_AXIS)

        # (16 x 16 x 8N)
        dec4 = self._add_decoding_layer(filter_count, False, dec3)
        dec4 = concatenate([dec4, enc5], axis=self.CONCATENATE_AXIS)

        # (32 x 32 x 8N)
        dec5 = self._add_decoding_layer(filter_count, False, dec4)
        dec5 = concatenate([dec5, enc4], axis=self.CONCATENATE_AXIS)
        
        # (64 x 64 x 4N)
        filter_count = first_layer_filter_count*4
        dec6 = self._add_decoding_layer(filter_count, False, dec5)
        dec6 = concatenate([dec6, enc3], axis=self.CONCATENATE_AXIS)

        # (128 x 128 x 2N)
        filter_count = first_layer_filter_count*2
        dec7 = self._add_decoding_layer(filter_count, False, dec6)
        dec7 = concatenate([dec7, enc2], axis=self.CONCATENATE_AXIS)

        # (256 x 256 x N)
        filter_count = first_layer_filter_count
        dec8 = self._add_decoding_layer(filter_count, False, dec7)
        dec8 = concatenate([dec8, enc1], axis=self.CONCATENATE_AXIS)

        # (512 x 512 x output_channel_count)
        dec9 = Activation(activation='relu')(dec8)
        dec9 = Conv2DTranspose(output_channel_count, self.DECONV_FILTER_SIZE, strides=self.DECONV_STRIDE)(dec9)
        dec9 = Activation(activation='sigmoid')(dec9)

        self.UNET = Model(input=inputs, output=dec9)

    def _add_encoding_layer(self, filter_count, sequence):
        new_sequence = LeakyReLU(0.2)(sequence)
        new_sequence = ZeroPadding2D(self.CONV_PADDING)(new_sequence)
        new_sequence = Conv2D(filter_count, self.CONV_FILTER_SIZE, strides=self.CONV_STRIDE)(new_sequence)
        new_sequence = BatchNormalization()(new_sequence)
        return new_sequence

    def _add_decoding_layer(self, filter_count, add_drop_layer, sequence):
        new_sequence = Activation(activation='relu')(sequence)
        new_sequence = Conv2DTranspose(filter_count, self.DECONV_FILTER_SIZE, strides=self.DECONV_STRIDE,
                                       kernel_initializer='he_uniform')(new_sequence)
        new_sequence = BatchNormalization()(new_sequence)
        if add_drop_layer:
            new_sequence = Dropout(0.5)(new_sequence)
        return new_sequence

    def get_model(self):
        return self.UNET

### その他関数定義
---

In [ ]:
IMAGE_SIZE = 512
TRAIN_PERCENTAGE = 0.8

# 値を-1から1に正規化する関数
def normalize_x(image):
    # image = image/127.5 - 1
    image = (image-np.min(image)) / (np.max(image)/2) - 1
    return image


# 値を0から1に正規化する関数
def normalize_y(image):
    image = image/255
    return image


# 値を0から255に戻す関数
def denormalize_y(image):
    image = image*255
    return image


# インプット画像を読み込む関数
def load_X(folder_path):
    import glob, SimpleITK
    from natsort import natsorted
    
    mhd_files = natsorted(glob.glob(folder_path+'*.mhd'))
    images = SimpleITK.GetArrayFromImage(SimpleITK.ReadImage(mhd_files[0]))
    
    image_files = []
    for i in range(images.shape[0]):
        image_files.append(mhd_files[0].split('/')[-1].split('.')[0] +'-'+str(i))    
    
    
    for mhd_name in mhd_files[1:int(len(mhd_files)*TRAIN_PERCENTAGE)]:
        mhd_array = SimpleITK.GetArrayFromImage(SimpleITK.ReadImage(mhd_name))
        images = np.concatenate([images,mhd_array])
        for i in range(mhd_array.shape[0]):
            image_files.append(mhd_name.split('/')[-1].split('.')[0] +'-'+str(i))     
    
    images = normalize_x(images[:, :, :, np.newaxis])
    
    #images = np.zeros((len(image_files), IMAGE_SIZE, IMAGE_SIZE, 1), np.float32)
    return images, image_files


# ラベル画像を読み込む関数
def load_Y(folder_path):
    import glob, SimpleITK
    from natsort import natsorted
    
    mhd_files = natsorted(glob.glob(folder_path+'*/*[!2].mhd'))
    images = SimpleITK.GetArrayFromImage(SimpleITK.ReadImage(mhd_files[0]))
    for mhd_name in mhd_files[1:int(len(mhd_files)*TRAIN_PERCENTAGE)]:
        mhd_array = SimpleITK.GetArrayFromImage(SimpleITK.ReadImage(mhd_name))
        images = np.concatenate([images,mhd_array])
    
    # GTの値の範囲が0-6だったので、1以上は1にしている
    images = np.where(images >= 1 , 1 ,images)
    return images[:, :, :, np.newaxis]


### train/test部分
---

In [ ]:
import os
import numpy as np
from keras.optimizers import Adam
import keras.backend as K
from keras.callbacks import ModelCheckpoint, EarlyStopping
# from unet import UNet

# ダイス係数を計算する関数
def dice_coef(y_true, y_pred):
    y_true = K.flatten(y_true)
    y_pred = K.flatten(y_pred)
    intersection = K.sum(y_true * y_pred)
    return 2.0 * intersection / (K.sum(y_true) + K.sum(y_pred) + 1)


# ロス関数
def dice_coef_loss(y_true, y_pred):
    return 1.0 - dice_coef(y_true, y_pred)


# U-Netのトレーニングを実行する関数
def train_unet():
    # trainingDataフォルダ配下にleft_imagesフォルダを置いている
    X_train, file_names = load_X('../Preprocess/mhd/')
    # trainingDataフォルダ配下にleft_groundTruthフォルダを置いている
    Y_train = load_Y('../Preprocess/GroundTruth/')

    # 入力はBGR3チャンネル
    input_channel_count = 1
    # 出力はグレースケール1チャンネル
    output_channel_count = 1
    # 一番初めのConvolutionフィルタ枚数は64
    first_layer_filter_count = 64
    # U-Netの生成
    
    print(X_train.shape, Y_train.shape)
    
    network = UNet(input_channel_count, output_channel_count, first_layer_filter_count)
    model = network.get_model()
    model.compile(loss=dice_coef_loss, optimizer=Adam(), metrics=[dice_coef])
    
    model.summary()

    BATCH_SIZE = 12
    # 20エポック回せば十分
    NUM_EPOCH = 20
    history = model.fit(X_train, Y_train, batch_size=BATCH_SIZE, epochs=NUM_EPOCH, verbose=1)
    model.save_weights('unet_weights.hdf5')


# 学習後のU-Netによる予測を行う関数
def predict():
    import cv2

    # testDataフォルダ配下にleft_imagesフォルダを置いている
    X_test, file_names = load_X('../Preprocess/mhd/')

    input_channel_count = 1
    output_channel_count = 1
    first_layer_filter_count = 64
    network = UNet(input_channel_count, output_channel_count, first_layer_filter_count)
    model = network.get_model()
    model.load_weights('unet_weights.hdf5')
    BATCH_SIZE = 12
    Y_pred = model.predict(X_test, BATCH_SIZE)

    for i, y in enumerate(Y_pred):
        # testDataフォルダ配下にleft_imagesフォルダを置いている
        save_path = 'U-Net_DataSet/test/input/'
        if os.path.exists(save_path): os.mkdir(save_path)
        img = cv2.imread(save_path + file_names[i]+".png")
        y = cv2.resize(y, (img.shape[1], img.shape[0]))
        cv2.imwrite('prediction' + str(i) + '.png', denormalize_y(y))


if __name__ == '__main__':
    train_unet()
#     predict()

(2308, 512, 512, 1) (2308, 512, 512, 1)


/home/dlbox/anaconda3/envs/TF2-NT/lib/python3.6/site-packages/ipykernel_launcher.py:93: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("ac...)`


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_7 (InputLayer)            (None, 512, 512, 1)  0                                            
__________________________________________________________________________________________________
zero_padding2d_53 (ZeroPadding2 (None, 514, 514, 1)  0           input_7[0][0]                    
__________________________________________________________________________________________________
conv2d_53 (Conv2D)              (None, 256, 256, 64) 1088        zero_padding2d_53[0][0]          
__________________________________________________________________________________________________
leaky_re_lu_47 (LeakyReLU)      (None, 256, 256, 64) 0           conv2d_53[0][0]                  
__________________________________________________________________________________________________
zero_paddi

In [ ]:
!ls ../Preprocess/mhd/*.mhd

In [ ]:
!ls ../Preprocess/GroundTruth/*/*.mhd

In [ ]:
import SimpleITK

m = SimpleITK.ReadImage('../Preprocess/GroundTruth/L0/okada1.mhd')

# print(m)
print("mhd Shape >>", SimpleITK.GetArrayFromImage(m).shape)

In [ ]:
os.listdir('../Preprocess/mhd/')[0]

In [ ]:
!ls ./U-Net_DataSet/train/input/L0.mhd